In [15]:
from langchain_community.chat_models import ChatTongyi
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [16]:
embedding_model_name = "/root/autodl-tmp/web/BAAI_bge"

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"device": "cuda"},  # 没 GPU 改成 "cpu"
    encode_kwargs={
        "normalize_embeddings": True,  # bge 必须！
    },
)

vectorstore = FAISS.load_local(
    folder_path="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)


In [17]:
query_list = ["借款人去世，继承人是否应履行偿还义务？ ","如何通过法律手段应对民间借贷纠纷？ ","没有赡养老人就无法继承财产吗？ ","""谁可以申请撤销监护人的监护资格？""","""你现在是一个精通中国法律的法官，请对以下案件做出分析：经
审理查明：被告人xxx于2017年12月，多次在本市xxx盗窃财物。具
体事实如下：（一）2017年12月9日15时许，被告人xxx在xxx店内，
盗窃白色毛衣一件（价值人民币259元）。现赃物已起获并发还。（二）
2017 年 12 月9日16时许，被告人xx在本市xxx店内，盗窃米白色大
衣一件（价值人民币1199元）。现赃物已起获并发还。（三）2017年1
2 月11 日19时许，被告人xxx在本市xxx内，盗窃耳机、手套、化妆
镜等商品共八件（共计价值人民币 357.3 元）。现赃物已起获并发还。
（四）2017 年12月11 日20时许，被告人xx在本市xxxx内，盗窃橙
汁、牛肉干等商品共四件（共计价值人民币 58.39 元）。现赃物已起获
并发还。2017 年12月11 日，被告人xx被公安机关抓获，其到案后如
实供述了上述犯罪事实。经鉴定，被告人xxx被诊断为精神分裂症，限
制刑事责任能力，有受审能力。""","""你现在是一个精通中国法律的法官，请对以下案件做出分析：2
012 年 5月1日，原告xxx在被告xxxx购买“玉兔牌”香肠15包，其
中价值558.6元的14包香肠已过保质期。xxx到收银台结账后，即径直
到服务台索赔，后因协商未果诉至法院，要求xxxx店支付14包香肠售
价十倍的赔偿金5586元。"""]
docs_list=[]
for query in query_list:
    docs= vectorstore.similarity_search(
    query=query,
    k=5)
    docs_list.append(docs)

In [18]:
def format_docs(docs):
    return "\n\n".join(
        f"[Chunk {i}]\n{doc.page_content}"
        for i, doc in enumerate(docs)
    )



prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "你是专业的法律知识问答助手。你需要使用以下检索到的上下文片段来回答问题，禁止根据常识和已知信息回答问题。如果你不知道答案，直接回答“未找到相关答案"
    ),
    (
        "human",
        "Context:\n{context}\n\nQuestion:\n{question}"
    ),
])
llm = ChatTongyi(
    model="qwen3-max",   
    api_key="你的api",
    temperature=0.0,
)
for query,context in zip(query_list,docs_list):
    final_prompt = prompt.format_messages(
    context=format_docs(context),
    question=query,
)
    response = llm.invoke(final_prompt)
    print(response.content)
    


借款人去世后，继承人是否应履行偿还义务，取决于其是否继承遗产。根据相关法律规定，债务以借款人的遗产实际价值为限进行清偿。如果继承人接受继承，则应在所继承遗产的实际价值范围内承担偿还义务；超过遗产实际价值的部分，继承人无需偿还，除非其自愿偿还。

此外，若继承人在被继承人死亡后、遗产处理前明确表示放弃继承，则该放弃行为具有法律效力，此时继承人不承担偿还义务。
未找到相关答案
未找到相关答案。
未找到相关答案
根据提供的案件事实及相关法律规定，现就本案作如下法律分析：

一、关于犯罪事实的认定  
被告人xxx于2017年12月多次实施盗窃行为，共四起，盗窃财物总价值为人民币1873.69元（259 + 1199 + 357.3 + 58.39）。根据《中华人民共和国刑法》第二百六十四条之规定，盗窃公私财物，数额较大的，或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的，构成盗窃罪。  
虽然本案中单次盗窃金额均未达到“数额较大”标准（通常司法解释中“数额较大”起点为1000元至3000元，各地略有差异），但被告人实施了**四次盗窃行为**，符合“多次盗窃”的情形，依法已构成盗窃罪。

二、关于刑事责任能力  
经司法鉴定，被告人被诊断为**精神分裂症，限制刑事责任能力**，且具有受审能力。依据《中华人民共和国刑法》第十八条第三款之规定：“尚未完全丧失辨认或者控制自己行为能力的精神病人犯罪的，应当负刑事责任，但是可以从轻或者减轻处罚。”  
因此，被告人应对其盗窃行为承担刑事责任，但依法**可以从轻或减轻处罚**。

三、关于量刑情节  
1. **如实供述**：被告人到案后如实供述全部犯罪事实，符合《中华人民共和国刑法》第六十七条第三款之规定，构成坦白，**可以从轻处罚**。  
2. **赃物全部追回并发还**：表明其社会危害性相对较小，可作为酌定从轻情节予以考虑。  
3. **限制刑事责任能力**：如前所述，依法可以从轻或减轻处罚。

四、综合判断  
被告人xxx的行为已构成盗窃罪，依法应在“三年以下有期徒刑、拘役或者管制，并处或者单处罚金”的法定刑幅度内量刑。鉴于其系**限制刑事责任能力人**、具有**坦白情节**、**赃物全部追回**，且无前科劣迹（假设无其他加重情节），依法可对其**从轻或减轻处罚**，并可考虑适用缓刑（若符合缓刑条件）。

综上，建议在量刑时

In [19]:
for query,context in zip(query_list,docs_list):
    final_prompt = prompt.format_messages(
    context="",
    question=query,
)
    response = llm.invoke(final_prompt)
    print(response.content)
    


根据检索到的上下文信息，未找到相关答案。
未找到相关答案
未找到相关答案
根据《中华人民共和国民法典》第三十六条规定，以下主体可以向人民法院申请撤销监护人的监护资格：

1. 被监护人住所地的居民委员会、村民委员会；
2. 学校、医疗机构、妇女联合会、残疾人联合会、未成年人保护组织、依法设立的老年人组织；
3. 民政部门；
4. 有关个人或组织（在上述单位未及时申请的情况下，民政部门应当向人民法院申请）。

这些主体在发现监护人存在严重损害被监护人身心健康、怠于履行监护职责、无法履行监护职责且拒绝将监护职责部分或者全部委托给他人，导致被监护人处于危困状态，或者其他严重侵害被监护人合法权益的情形时，可以依法申请撤销其监护资格。
根据提供的案件事实及相关法律规定，现就本案作如下法律分析：

一、行为定性：构成盗窃罪

被告人xxx在2017年12月期间，先后四次在不同商店内秘密窃取他人财物，其行为符合《中华人民共和国刑法》第二百六十四条关于盗窃罪的构成要件。该条规定：“盗窃公私财物，数额较大的，或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金。”

值得注意的是，虽然被告人所盗财物总价值为人民币1873.69元，尚未达到“数额较大”（司法实践中通常以人民币1000元至3000元以上为起点，具体标准依地方规定而定），但其在短时间内实施了四次盗窃行为，属于“多次盗窃”情形。根据最高人民法院、最高人民检察院《关于办理盗窃刑事案件适用法律若干问题的解释》第三条第一款规定：“二年内盗窃三次以上的，应当认定为‘多次盗窃’。”因此，即使每次盗窃金额较小，亦可构成盗窃罪。

二、刑事责任能力：限制刑事责任能力人

经司法鉴定，被告人被诊断为精神分裂症，属于限制刑事责任能力人，但具备受审能力。根据《中华人民共和国刑法》第十八条第三款规定：“尚未完全丧失辨认或者控制自己行为能力的精神病人犯罪的，应当负刑事责任，但是可以从轻或者减轻处罚。”因此，对被告人应当追究刑事责任，但依法可以从轻或减轻处罚。

三、量刑情节分析

1. **坦白情节**：被告人到案后如实供述全部犯罪事实，符合《刑法》第六十七条第三款关于“如实供述自己罪行”的规定，可依法从轻处罚。

2. **赃物已全部起获并发还**：表明社会危害性相对较小，被害方损失已挽回，可作为酌定从